In [ ]:
import csv
import graphviz
from automata_toolkit import regex_to_nfa, nfa_to_dfa, dfa_to_efficient_dfa, dfa_to_regex, visual_utils
import copy
import networkx as nx
from greenery import parse
import os


def automata_product(A, B, alphabet):
    # construct the product automaton of A and B through networkx (easier to work with)
    # given the states of A and B, states of product are (s_A x s_B)
    # transitions between s_1 and s_2 if s_1[0] -> s_2[0] in A and s_1[1] -> s_2[1] in B
    # initial state is (s_A0, s_B0)
    # final states are (s_Af, s_Bf) where s_Af is a final state in A and s_Bf is a final state in B
    prod = nx.DiGraph()
    prod.add_nodes_from([(state_A, state_B) for state_A in A['states'] for state_B in B['states']])   
    found_start = False
    checked_all_once = False
    end_nodes = []
    for u in prod.nodes():
        for v in prod.nodes():
            for symbol in alphabet:
                if symbol in A['transition_function'][u[0]] and A['transition_function'][u[0]][symbol] == v[0] and symbol in B['transition_function'][u[1]] and B['transition_function'][u[1]][symbol] == v[1]:
                    prod.add_edge(u, v, label=symbol)
            if not found_start and u[0] == A['initial_state'] and u[1] == B['initial_state']:
                start_node = u
                found_start = True
            if not checked_all_once and v[0] in A['final_states'] and v[1] in B['final_states']:
                end_nodes.append(v)
        checked_all_once = True
    return prod, start_node, end_nodes

def is_nonempty(A, start_node, end_nodes):
    #check if automaton A (represented as a networkX directed graph) is non-empty
    #equates to see if there is a path from the initial state to a final state
    for end_node in end_nodes:
        if nx.has_path(A, start_node, end_node):
            return True
    return False

def step_one(original_regexp):
    # convert original regular expression to NFA
    nfa = regex_to_nfa.regex_to_nfa(original_regexp)
    # convert NFA to DFA
    dfa = nfa_to_dfa.nfa_to_dfa(nfa)
    # convert DFA to efficient DFA A
    return dfa_to_efficient_dfa.dfa_to_efficient_dfa(dfa)

def step_two(A, new_regexps):
    # construct A' from A
    A_prime = copy.deepcopy(A)
    # final states in A' are the states in A that are not final
    A_prime['final_states'] = [state for state in A_prime['states'] if state not in A['final_states']] 
    A_prime['final_reachable_states'] = [state for state in A_prime['final_states'] if state in A_prime['reachable_states']]
    # change the alphabet of A' to be e0, ..., ek
    symbol_to_regex = {f'e{i+1}':regexp for i, regexp in enumerate(new_regexps)}
    A_prime['alphabets'] = list(symbol_to_regex.keys())
    # empty the transition function of A'
    A_prime['transition_function'] = {state: {} for state in A_prime['states']}
    # change the transitions of A'
    A_temp = copy.deepcopy(A)
    for state_i, transitions in A['transition_function'].items():
        # for each pair of states, construct a new automaton equal to A but with initial state state_i and final state state_j
        A_temp['initial_state'] = state_i
        for state_j in A['states']:
            A_temp['final_states'] = [state_j]
            A_temp['final_reachable_states'] = [state_j if state_j in A_temp['reachable_states'] else []]
            # for each regexp, construct an automaton and check if the product between it and A_temp is empty
            for symbol in A_prime['alphabets']:
                A_regexp = step_one(symbol_to_regex[symbol])
                #check for non-emptiness of the product between A_temp and A_regexp
                alphabet =  list(set(A_temp['alphabets']) & set( A_regexp['alphabets']))
                A_product, start_node, end_nodes = automata_product(A_temp, A_regexp, alphabet)
                # if non-empty, add a transition from state_i to state_j with label e_i
                if is_nonempty(A_product, start_node, end_nodes):
                    A_prime['transition_function'][state_i][symbol] = state_j
    return A_prime, symbol_to_regex

def step_three(A_prime):
    #returns the complement automaton of A_prime and its relative regular expression
    # construct the complement automaton of A_prime
    A_complement = copy.deepcopy(A_prime)
    A_complement['final_states'] = [state for state in A_complement['states'] if state not in A_prime['final_states']]
    A_complement['final_reachable_states'] = [state for state in A_complement['final_states'] if state in A_complement['reachable_states']]
    return A_complement  

def translate_automata_to_regex(A, symbol_to_regex):
    #given an automaton A and a dictionary symbol_to_regex, returns the regular expression of A
    #substitute the symbols of any transitions with their relative regexps
    previous_transition_function = copy.deepcopy(A['transition_function'])
    A['alphabets'] = list(symbol_to_regex.values())
    A['transition_function'] = {state: {} for state in A['states']}
    for state, transitions in previous_transition_function.items():
        for symbol, next_state in transitions.items():
            A['transition_function'][state][symbol_to_regex[symbol]] = next_state
    # construct the regular expression of A
    A_regex = dfa_to_regex.dfa_to_regex(A)
    return A_regex

def generate_strings(parsed, length):
    # generate a set of strings of length up to length from a parsed regular expression
    strings = set()
    for string in parsed.strings():
        if len(string) > length:
            break
        strings.add(string)
    return strings

def count_coverage(a, b):
    # count the number of strings in a that are also in b
    return len(a & b)/len(a)

def count_verbosity(a, b):
    # count the number of strings in b that are not in a
    return len(b - a)/len(b)

def check_language_coverage(orig_regexp, new_regexp, length=20):
    # check how far the language covered by the new regexp is from the original one, up to a certain string length
    # convert the + sign to the | sign
    orig_regexp = orig_regexp.replace('+', '|')
    new_regexp = new_regexp.replace('+', '|')
    orig_parsed = parse(orig_regexp)
    new_parsed = parse(new_regexp)
    orig_lang = generate_strings(orig_parsed, length)
    new_lang = generate_strings(new_parsed, length)
    return count_coverage(orig_lang, new_lang), count_verbosity(orig_lang, new_lang)

In [ ]:
original_regexp, new_regexps = "a(ba+c)*",["a","ac*b","c"]
A = step_one(original_regexp)
visual_utils.draw_dfa(A, "A_d")
#construct A' from A
A_prime, symbol_to_regex = step_two(A, new_regexps)
visual_utils.draw_dfa(A_prime, "A'")
A_complement = step_three(A_prime)
visual_utils.draw_dfa(A_complement, "Complement of A'")
A_complement_regexp = translate_automata_to_regex(A_complement, symbol_to_regex)
print(f'Original regular expression: {original_regexp}')
print(f'Regular expressions to be replaced: {new_regexps}')
print(f'New regular expression: {A_complement_regexp}')
coverage, verbosity = check_language_coverage(original_regexp, A_complement_regexp, length=5)
print(f'Coverage of the new regular expression wrt. the old one: {coverage}')
print(f'Verbosity of the new regular expression wrt. the old one: {verbosity}')
